# LoRA Fine-Tuning Notebook

In [1]:
# import necessary python libraries
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, pipeline
from torch.utils.data import Dataset
import torch
import pickle
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import time
import math
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import json
from pathlib import Path
import random
import copy

In [2]:
""" configs = [
    {
        "chunk_filename_pkl": "./data/kahneman_chunks.pkl",
        "model_name": "deepseek-ai/deepseek-llm-7b-base",
        "r": 64,
        "lora_alpha": 128,
        "lora_dropout": 0.05,
        "target_modules": [
            "q_proj",
            "v_proj",
            "k_proj",
            "o_proj",
            "gate_proj"
        ],
        "bias": "none",
        "task_type": "CAUSAL_LM",
        "output_dir": "./lora_finetuned_model/deepseek-llm-7b-base/LoRA_deep_book_run",
        "per_device_train_batch_size": 8,
        "gradient_accumulation_steps": 4,
        "learning_rate": 5e-05,
        "num_train_epochs": 5,
        "save_strategy": "epoch",
        "bf16": True,
        "logging_steps": 10,
        "report_to": "none"
    },
    {
        "chunk_filename_pkl": "./data/kahneman_chunks.pkl",
        "model_name": "deepseek-ai/deepseek-llm-7b-base",
        "r": 16,
        "lora_alpha": 32,
        "lora_dropout": 0.1,
        "target_modules": [
            "q_proj",
            "v_proj"
        ],
        "bias": "none",
        "task_type": "CAUSAL_LM",
        "output_dir": "./lora_finetuned_model/deepseek-llm-7b-base/LoRA_shallow_book_run",
        "per_device_train_batch_size": 8,
        "gradient_accumulation_steps": 2,
        "learning_rate": 0.0002,
        "num_train_epochs": 3,
        "save_strategy": "epoch",
        "bf16": True,
        "logging_steps": 10,
        "report_to": "none"
    }
] """

' configs = [\n    {\n        "chunk_filename_pkl": "./data/kahneman_chunks.pkl",\n        "model_name": "deepseek-ai/deepseek-llm-7b-base",\n        "r": 64,\n        "lora_alpha": 128,\n        "lora_dropout": 0.05,\n        "target_modules": [\n            "q_proj",\n            "v_proj",\n            "k_proj",\n            "o_proj",\n            "gate_proj"\n        ],\n        "bias": "none",\n        "task_type": "CAUSAL_LM",\n        "output_dir": "./lora_finetuned_model/deepseek-llm-7b-base/LoRA_deep_book_run",\n        "per_device_train_batch_size": 8,\n        "gradient_accumulation_steps": 4,\n        "learning_rate": 5e-05,\n        "num_train_epochs": 5,\n        "save_strategy": "epoch",\n        "bf16": True,\n        "logging_steps": 10,\n        "report_to": "none"\n    },\n    {\n        "chunk_filename_pkl": "./data/kahneman_chunks.pkl",\n        "model_name": "deepseek-ai/deepseek-llm-7b-base",\n        "r": 16,\n        "lora_alpha": 32,\n        "lora_dropout": 0.

In [2]:
random_seeds = [3, 24, 34, 46, 59, 60, 61, 62, 64, 67, 68, 73, 74, 78, 81, 83, 88, 92, 94, 96]

In [3]:
# Model
model_name = "deepseek-ai/deepseek-llm-7b-base"

# files
chunk_filename_pkl = "./data/kahneman_chunks.pkl" # in this case a pickle file with a list of paragraphs stored in it


# LoRA
r = 16
lora_alpha = 32
lora_dropout = 0.1
target_modules = ["q_proj", "v_proj"]
bias = "none"
task_type = "CAUSAL_LM"

# Training
output_dir = "./lora_finetuned_model/deepseek-llm-7b-base/LoRA_shallow_book_run"
per_device_train_batch_size = 8
gradient_accumulation_steps = 2
learning_rate = 0.0002
num_train_epochs = 3
save_strategy = "epoch"
bf16 = True
logging_steps = 10
report_to = "none"

In [ ]:
""" # Model
model_name = "deepseek-ai/deepseek-llm-7b-base"

# files
chunk_filename_pkl = "./data/kahneman_chunks.pkl" # in this case a pickle file with a list of paragraphs stored in it


# LoRA
r = 64
lora_alpha = 128
lora_dropout = 0.05
target_modules = ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj"]
bias = "none"
task_type = "CAUSAL_LM"

# Training
output_dir = "./lora_finetuned_model/deepseek-llm-7b-base/LoRA_deep_book_run"
per_device_train_batch_size = 8
gradient_accumulation_steps = 4
learning_rate = 5e-5
num_train_epochs = 5
save_strategy = "epoch"
bf16 = True
logging_steps = 10
report_to = "none" """

In [4]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
original_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Load the pickle file
with open(chunk_filename_pkl, 'rb') as f:
    kahneman_paragraphs = pickle.load(f)

# Ensure it's a list
if not isinstance(kahneman_paragraphs, list):
    raise ValueError("The loaded pickle object is not a list!")

print(f"Loaded {len(kahneman_paragraphs)} paragraphs from pickle.")

Loaded 839 paragraphs from pickle.


In [6]:
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.examples = []

        for text in texts:
            encoding = tokenizer(
                text,
                truncation=True,
                padding="max_length",
                max_length=max_length,
                return_tensors="pt"
            )

            input_ids = encoding["input_ids"].squeeze()
            attention_mask = encoding["attention_mask"].squeeze()

            # Set labels the same as input_ids, but ignore padding with -100
            labels = input_ids.clone()
            labels[labels == tokenizer.pad_token_id] = -100

            self.examples.append({
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "labels": labels
            })

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]


# Create dataset
dataset = TextDataset(kahneman_paragraphs, tokenizer)

In [7]:
for seed in random_seeds:
    # Define LoRA configuration
    lora_config = LoraConfig(
        r=r,  # Rank: Controls adaptation capacity
        lora_alpha=lora_alpha,  # Scaling factor
        lora_dropout=lora_dropout,  # Dropout probability
        target_modules=target_modules,  # Target attention layers
        bias=bias,
        task_type=task_type,
    )

    # Apply LoRA to the model
    model = get_peft_model(original_model, lora_config)
    model.print_trainable_parameters()  # Verify trainable params

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  # Causal LM, not masked LM
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        save_strategy=save_strategy,
        bf16=bf16,
        logging_steps=logging_steps,
        report_to=report_to,  # Disable logging to external services like WandB
        seed=seed
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=data_collator,
    )

    # Record the start time
    start_time = time.time()

    # Train the model
    trainer.train()

    # Record the end time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = (end_time - start_time)/60

    # Print the training duration in seconds
    print(f"Training took {elapsed_time:.2f} minutes.")

    # Save the LoRA fine-tuned model
    model.save_pretrained(output_dir + str(seed))
    tokenizer.save_pretrained(output_dir + str(seed))

trainable params: 7,864,320 || all params: 6,918,230,016 || trainable%: 0.1137


Step,Training Loss
10,2.613700
20,2.592700
30,2.531000
40,2.486900
50,2.488700
60,2.460500
70,2.450500
80,2.408600
90,2.435400
100,2.441600


Training took 3.15 minutes.
trainable params: 7,864,320 || all params: 6,918,230,016 || trainable%: 0.1137


Step,Training Loss
10,2.598000
20,2.552500
30,2.504700
40,2.547400
50,2.482300
60,2.489500
70,2.447200
80,2.414200
90,2.417600
100,2.427900


Training took 3.17 minutes.
trainable params: 7,864,320 || all params: 6,918,230,016 || trainable%: 0.1137


...

Training took 3.16 minutes.
trainable params: 7,864,320 || all params: 6,918,230,016 || trainable%: 0.1137


Step,Training Loss
10,2.667800
20,2.519800
30,2.532800
40,2.512300
50,2.459700
60,2.452700
70,2.448300
80,2.429700
90,2.426700
100,2.431400


Training took 3.16 minutes.
trainable params: 7,864,320 || all params: 6,918,230,016 || trainable%: 0.1137


Step,Training Loss
10,2.629600
20,2.524400
30,2.538900
40,2.501300
50,2.491200
60,2.489900
70,2.425200
80,2.482900
90,2.398800
100,2.407700


Training took 3.16 minutes.


In [ ]:
""" # Define LoRA configuration
lora_config = LoraConfig(
    r=r,  # Rank: Controls adaptation capacity
    lora_alpha=lora_alpha,  # Scaling factor
    lora_dropout=lora_dropout,  # Dropout probability
    target_modules=target_modules,  # Target attention layers
    bias=bias,
    task_type=task_type,
)

# Apply LoRA to the model
model = get_peft_model(original_model, lora_config)
model.print_trainable_parameters()  # Verify trainable params """

trainable params: 91,914,240 || all params: 7,002,279,936 || trainable%: 1.3126


In [ ]:
""" data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  # Causal LM, not masked LM """

In [9]:
""" # Generate 20 unique random seeds
random_seeds = random.sample(range(1_000_000), 20)

# run trainings loop with 20 different configs
for config_idx, config in enumerate(configs):
    for run_idx, run_seed in enumerate(random_seeds):
        config = copy.deepcopy(config)
        config["seed"] = run_seed


        # Create a unique output directory
        base_output = Path(config["output_dir"])
        run_output = base_output.parent / f"{base_output.name}_seed{run_seed}"
        run_output.mkdir(parents=True, exist_ok=True)
        config["output_dir"] = str(run_output)

        print(f"\n=== Starting run {run_idx} for config {config_idx} with seed {run_seed} ===")
        print(f"Output dir: {config['output_dir']}")
        print(f"Num train epochs: {config['num_train_epochs']} ({type(config['num_train_epochs'])})")
        
        # Define LoRA configuration
        lora_config = LoraConfig(
            r=config["r"],  # Rank: Controls adaptation capacity
            lora_alpha=config["lora_alpha"],  # Scaling factor
            lora_dropout=config["lora_dropout"],  # Dropout probability
            target_modules=config["target_modules"],  # Target attention layers
            bias=config["bias"],
            task_type=config["task_type"],
        )

        # Apply LoRA to the model
        model = get_peft_model(original_model, lora_config)
        model.print_trainable_parameters()  # Verify trainable params

        # Prepare TrainingArguments
        training_args = TrainingArguments(
            output_dir=config["output_dir"],
            per_device_train_batch_size=config["per_device_train_batch_size"],
            gradient_accumulation_steps=config["gradient_accumulation_steps"],
            learning_rate=config["learning_rate"],
            num_train_epochs=config["num_train_epochs"],
            save_strategy=config["save_strategy"],
            bf16=config["bf16"],
            logging_steps=config["logging_steps"],
            report_to=config["report_to"],
            dataloader_drop_last=True,
        )

        # Initialize Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset,
            data_collator=data_collator,
        )
        print(f"len(dataset): {len(dataset)}")
        print(f"per_device_train_batch_size: {training_args.per_device_train_batch_size}")
        print(f"gradient_accumulation_steps: {training_args.gradient_accumulation_steps}")

        effective_batch_size = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
        steps_per_epoch = math.ceil(len(dataset) / effective_batch_size)

        print(f"Effective batch size: {effective_batch_size}")
        print(f"Steps per epoch (Trainer internal): {steps_per_epoch}")
        print(f"Expected total steps: {config['num_train_epochs'] * steps_per_epoch}")


        effective_batch_size = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
        steps_per_epoch = len(dataset) // effective_batch_size
        print(f"Trainer uses steps_per_epoch = {steps_per_epoch}")
        print(f"Total steps = {config['num_train_epochs'] * steps_per_epoch} = {config['num_train_epochs']} epochs * {steps_per_epoch} steps")
        print(f"Leftover samples per epoch: {len(dataset) % effective_batch_size}")


        # Record the start time
        start_time = time.time()

        # Train the model
        trainer.train()

        print(f"Trainer state: {trainer.state}")

        # Record the end time and compute duration
        end_time = time.time()
        elapsed_time = (end_time - start_time) / 60
        print(f"Training took {elapsed_time:.2f} minutes.") """

' # Generate 20 unique random seeds\nrandom_seeds = random.sample(range(1_000_000), 20)\n\n# run trainings loop with 20 different configs\nfor config_idx, config in enumerate(configs):\n    for run_idx, run_seed in enumerate(random_seeds):\n        config = copy.deepcopy(config)\n        config["seed"] = run_seed\n\n\n        # Create a unique output directory\n        base_output = Path(config["output_dir"])\n        run_output = base_output.parent / f"{base_output.name}_seed{run_seed}"\n        run_output.mkdir(parents=True, exist_ok=True)\n        config["output_dir"] = str(run_output)\n\n        print(f"\n=== Starting run {run_idx} for config {config_idx} with seed {run_seed} ===")\n        print(f"Output dir: {config[\'output_dir\']}")\n        print(f"Num train epochs: {config[\'num_train_epochs\']} ({type(config[\'num_train_epochs\'])})")\n        \n        # Define LoRA configuration\n        lora_config = LoraConfig(\n            r=config["r"],  # Rank: Controls adaptation cap

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs,
    save_strategy=save_strategy,
    bf16=bf16,
    logging_steps=logging_steps,
    report_to=report_to  # Disable logging to external services like WandB
)

In [11]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

In [12]:
# Record the start time
start_time = time.time()

# Train the model
trainer.train()

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = (end_time - start_time)/60

# Print the training duration in seconds
print(f"Training took {elapsed_time:.2f} minutes.")

# In case your getting the following error: RuntimeError: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "/opt/conda/conda-bld/pytorch_1729647382455/work/c10/cuda/CUDACachingAllocator.cpp":995, please report a bug to PyTorch. 
# Your account doesn't fulfill the necessary GPU requirements. Keep in mind that due to self-attention the input length has quadratic costs
# You can check your VRAM and other GPU related metrics by typing nvidia-smi in the terminal

Step,Training Loss
10,2.622500
20,2.552200
30,2.516700
40,2.463900
50,2.442500
60,2.416000
70,2.363000
80,2.371900
90,2.323000
100,2.291000


Training took 5.11 minutes.


In [13]:
# Save the LoRA fine-tuned model
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./lora_finetuned_model/deepseek-llm-7b-base/LoRA_deep_book_run/tokenizer_config.json',
 './lora_finetuned_model/deepseek-llm-7b-base/LoRA_deep_book_run/special_tokens_map.json',
 './lora_finetuned_model/deepseek-llm-7b-base/LoRA_deep_book_run/tokenizer.json')

In [14]:
var_names = [
    "chunk_filename_pkl", "model_name",
    "r", "lora_alpha", "lora_dropout", "target_modules", "bias", "task_type",
    "output_dir", "per_device_train_batch_size", "gradient_accumulation_steps",
    "learning_rate", "num_train_epochs", "save_strategy",
    "bf16", "logging_steps", "report_to"
]

# Construct dictionary from current global variables
config = {var: globals()[var] for var in var_names}

# Save to JSON
with open(f"{output_dir}/experiment_config.json", "w") as f:
    json.dump(config, f, indent=4)

In [15]:
# Evaluate loss
trainer = Trainer(model=model)
eval_results = trainer.evaluate(dataset)
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

Perplexity: 9.65
